#### Download dataset, Takes a billion years~ Even if u run it the 2nd time.
Please comment it, if u ran once once before.

In [1]:
from torchtext.datasets import IWSLT2017
train_data, valid_data, test_data = IWSLT2017(language_pair=('en','de'))

#### Load dataset, so we stored in pickle.
If dataset is not loaded, Skil this line and reuse
The reason is that pickle is much faster than Downloading the entire dataset.

In [2]:
import pickle
def _load_dataset(filename):
    with open(f'.data/{filename}.pkl', 'rb') as f:
        train ,val, test = pickle.load(f)
    return train, val, test

try: 
    train_list, valid_list, test_list = _load_dataset('IWSLT2017_en_de')
    print("variables loaded successfully!")
    # file Loaded
except:
    train_list, valid_list, test_list = list(train_data), list(valid_data), list(test_data)
    train_copy, val_copy, test_copy = train_list.copy(), valid_list.copy(), test_list.copy() 
    print('Variables not loaded')

Variables not loaded


#### Loading tokenizers
Tokenizers are illustrated as just splitting the sentences 

YES It's different to trax.

- [Hello, How are u] -> ['Hello' , ',' , 'how', 'are', 'u']
- len = 1, len = 5

In [3]:
import spacy

"""Download these files in your shell"""
#python -m spacy download de_dep_news_trf
#python -m spacy download en_core_web_trf
#python -m spacy download fr_dep_news_trf
#python -m spacy download zh_core_web_trf
dict_spacy = {'en':'en_core_web_trf', 'de':'de_dep_news_trf', 'fr':'fr_dep_news_trf', 'cn': 'zh_core_web_trf', 
             'cat': 'ca_core_news_trf'}


# https://spacy.io/usage/models
# english, german, french, chinese, catalan
spacy_eng = spacy.load(dict_spacy['en'])
spacy_ger = spacy.load(dict_spacy['de'])

en_tokenizer = spacy_eng.tokenizer
de_tokenizer = spacy_ger.tokenizer


def _tokenize_en(sentence):
    return [tok.text.lower() for tok in en_tokenizer(sentence)] 
def _tokenize_de(sentence):
    return [tok.text.lower() for tok in de_tokenizer(sentence)] 

#### Process for creating vocab
Vocab is basically tokenize in trax
It is a dictionary where it maps string to int vice versa
- ['Hello', 'how' , 'are', 'u'] -> [342, 21, 64, 69]

In [4]:
from collections import Counter
from tqdm import tqdm

train_iter = iter(train_list)
counter_en = Counter()
counter_de = Counter()
for (src, trg) in tqdm(train_iter):
    counter_en.update(_tokenize_en(src)) # Update en for src
    counter_de.update(_tokenize_de(trg)) # Update de for trg

206112it [00:33, 6180.46it/s]


In [5]:
from torchtext.vocab import vocab, OrderedDict, build_vocab_from_iterator

vocab_en = vocab(OrderedDict(sorted(counter_en.items(), key=lambda x: x[1], reverse=True)),
                min_freq=2
                )
vocab_de = vocab(OrderedDict(sorted(counter_de.items(), key=lambda x: x[1], reverse=True)),
                min_freq=2
                )

In [6]:
def insert_special_token(token: str, index:int, vocab:vocab):
    if token not in vocab: vocab.insert_token(token, index)
    return vocab

special_tokens = ['<UNK>', '<BOS>', '<EOS>','<PAD>']
for i , token in enumerate(special_tokens):
    insert_special_token(token, i, vocab_en)
    insert_special_token(token, i, vocab_de)
    

# If token not found, The default index should be.
vocab_en.set_default_index(vocab_en['<UNK>'])
vocab_de.set_default_index(vocab_de['<UNK>'])

In [7]:
# Str to int
print('Str to int')
print(f"src:  <'UNK'>: {vocab_en['<UNK>']}, <'BOS'> : {vocab_en['<BOS>']}, <'EOS'> {vocab_en['<EOS>']}, <'PAD'> {vocab_en['<PAD>']}")
print(f"trg:  <'UNK'>: {vocab_de['<UNK>']}, <'BOS'> : {vocab_de['<BOS>']}, <'EOS'> {vocab_de['<EOS>']}, <'PAD'> {vocab_de['<PAD>']}")


#Int to str
print(f'\n\nInt to str : {vocab_en.lookup_token(0)}')


# Len of vocab
print('\n\nlen of vocab')
print(f"src_vocab_len: {len(vocab_en)} , trg_vocab_len: {len(vocab_de)}")

Str to int
src:  <'UNK'>: 0, <'BOS'> : 1, <'EOS'> 2, <'PAD'> 3
trg:  <'UNK'>: 0, <'BOS'> : 1, <'EOS'> 2, <'PAD'> 3


Int to str : <UNK>


len of vocab
src_vocab_len: 33667 , trg_vocab_len: 58554


### Dumping Variables in Pickle
Now that all the preprocessing is completed, we'll be dumping all the files into pickle as shown above.

- Train, val , test of "IWSLT2017_en_de" dataset
- vocab and tokenizers of en -> de [src -> trg]

In [8]:
from termcolor import colored

def _store_dataset(filename,train, val, test):
    with open(f'.data/{filename}.pkl', 'wb') as f:
        pickle.dump([train,val,test], f)

if not train_copy or not val_copy or not test_copy:
    class MyException(Exception):
        pass
    raise MyException(f"train_list is {train_list}. Please run this again ")
        
try: 
    _store_dataset(filename='IWSLT2017_en_de', train = train_copy, val = val_copy, test = test_copy)
except Exception as e: 
    msg = 'Failed to store dataset: \nif its ur 1st time running this, be warned \n'+ str(e)
    print(colored(msg, 'blue'))


with open(f'.data/en_to_de_vocab_token.pkl', 'wb') as f:
     pickle.dump([vocab_en, vocab_de, en_tokenizer, de_tokenizer], f)

In [9]:
src_transform = lambda x: [vocab_en['<BOS>']] + [vocab_en[token.text.lower()] for token in en_tokenizer(x)] + [vocab_en['<EOS>']]
trg_transform = lambda x: [vocab_en['<BOS>']] + [vocab_de[token.text.lower()] for token in de_tokenizer(x)] + [vocab_en['<EOS>']]

print("input to the text_transform:", "How are you")
print("output of the text_transform:", src_transform("How are you"))

input to the text_transform: How are you
output of the text_transform: [1, 68, 26, 16, 2]


In [10]:
import random
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 32  # A batch size of 8

def collate_batch(batch):
    src_list, trg_list = [], []
    for (src, trg) in batch:
        src_list.append(torch.tensor(src_transform(src)))
        trg_list.append(torch.tensor(trg_transform(trg)))
    return pad_sequence(src_list, padding_value=vocab_en['<PAD>']).cuda(), pad_sequence(trg_list, padding_value=vocab_de['<PAD>']).cuda()


def batch_sampler(dataset:list):
    indices = [(i, len(en_tokenizer(s[1]))) for i, s in enumerate(dataset)]
    random.shuffle(indices)
    pooled_indices = []
    # create pool of indices with similar lengths 
    for i in range(0, len(indices), batch_size * 100):
        pooled_indices.extend(sorted(indices[i:i + batch_size * 100], key=lambda x: x[1]))

    pooled_indices = [x[0] for x in pooled_indices]

    # yield indices for current batch
    for i in range(0, len(pooled_indices), batch_size):
        yield pooled_indices[i:i + batch_size]


### Spawn Dataloaders
Spawn dataloaders for train, validation and test sets

In [11]:
"""
from seq_len_batch_sampler import BySequenceLengthSampler

bucket_boundaries = [50,100,125,150,175,200,250,256]

train_sampler = BySequenceLengthSampler(valid_list,de_tokenizer,bucket_boundaries, batch_size)
val_sampler = BySequenceLengthSampler(valid_list,de_tokenizer,bucket_boundaries, batch_size)
test_sampler = BySequenceLengthSampler(valid_list,de_tokenizer,bucket_boundaries, batch_size)


train_loader = DataLoader(train_list,
                          collate_fn = collate_batch,
                          batch_sampler=train_sampler, 
                          num_workers=0
                         )
valid_loader = DataLoader(valid_list,
                          collate_fn = collate_batch,
                          batch_sampler=val_sampler, 
                          num_workers=0
                         )
test_loader = DataLoader(test_list,
                         collate_fn = collate_batch,
                         batch_sampler=test_sampler, 
                         num_workers=0
                        )
"""
train_loader = DataLoader(train_list, batch_sampler = batch_sampler(train_list), collate_fn=collate_batch)
valid_loader = DataLoader(valid_list, batch_size=batch_size, collate_fn=collate_batch, drop_last=True, shuffle=True)
test_loader = DataLoader(test_list, batch_size=batch_size, collate_fn=collate_batch, drop_last=True, shuffle=True)

In [12]:
# int to str
def _detokenize_en(tokens):
    return [vocab_en.lookup_token(token) for token in tokens]

def _detokenize_de(tokens):
    return [vocab_de.lookup_token(token) for token in tokens]

In [13]:
train_data_in_iter = next(iter(train_loader))
src, trg = train_data_in_iter[0], train_data_in_iter[1]


print(src[:,0])
print(trg[:,0])
print(_detokenize_en(src[:,0]))
print(_detokenize_de(trg[:,0]))


tensor([  1, 903, 260,   5,   2,   3,   3,   3,   3,   3,   3],
       device='cuda:0')
tensor([   1, 1234,  179,    6,    2,    3], device='cuda:0')
['<BOS>', 'sex', '!', '\n', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
['<BOS>', 'sex', '!', '\n', '<EOS>', '<PAD>']


#### Preprocessing before sending it to transformer
We want to feed in [Batch_size, sentence_len] to our transformer, therefore it's better to transpose it

In [14]:
# We want to feed in 
# [batch_size, sentence] 
print(f"Before transpose  \nsrc_shape {src.shape} \ntrg_shape {trg.shape}")
src_transposed = torch.transpose(src, dim0 = 1, dim1 = 0)
trg_transposed = torch.transpose(trg, dim0 = 1, dim1 = 0)
print(f"\n\nAfter transposed  \nsrc_shape {src_transposed.shape} \ntrg_shape {trg_transposed.shape}")

Before transpose  
src_shape torch.Size([11, 32]) 
trg_shape torch.Size([6, 32])


After transposed  
src_shape torch.Size([32, 11]) 
trg_shape torch.Size([32, 6])


In [15]:
# Compare with above, Ensuring the transposed did it's job properly.
print(src_transposed[0])
print(trg_transposed[0])
print(_detokenize_en(src_transposed[0]))
print(_detokenize_de(trg_transposed[0]))

tensor([  1, 903, 260,   5,   2,   3,   3,   3,   3,   3,   3],
       device='cuda:0')
tensor([   1, 1234,  179,    6,    2,    3], device='cuda:0')
['<BOS>', 'sex', '!', '\n', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
['<BOS>', 'sex', '!', '\n', '<EOS>', '<PAD>']


#### Memory Management

In [16]:
## BAD
#del src, trg, src_transposed, trg_transposed
#del train_data_in_iter
#torch.cuda.empty_cache()
#print(torch.cuda.memory_allocated())
#print(torch.cuda.memory_reserved())

"""Better"""
src,trg = src.cpu(), trg.cpu()
src_transposed,trg_transposed = src_transposed.cpu(),trg_transposed.cpu()
del src, trg, src_transposed, trg_transposed
torch.cuda.empty_cache()

### Transformer Model

In [17]:
from attention_transformer import Transformer
import torch.nn as nn
SRC_VOCAB_SIZE ,TRG_VOCAB_SIZE = len(vocab_en) , len(vocab_de)
SRC_PAD_IDX, TRG_PAD_IDX = vocab_en['<PAD>'] , vocab_de['<PAD>']
MAX_SENTENCE_LENGTH = 256
EMBED_SIZE , NUM_LAYERS , FORWARD_EXPANSION , HEADS = 256, 3, 2 , 8
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DROPOUT = 0.1

model = Transformer(
    src_vocab_len=SRC_VOCAB_SIZE,
    trg_vocab_len=TRG_VOCAB_SIZE,
    src_pad_idx = SRC_PAD_IDX,
    trg_pad_idx = TRG_PAD_IDX,
    src_max_sentence_len = MAX_SENTENCE_LENGTH,
    trg_max_sentence_len = MAX_SENTENCE_LENGTH,
    hid_dim = EMBED_SIZE,
    n_layers = NUM_LAYERS,
    n_heads = HEADS,
    ff_dim_multiplier = FORWARD_EXPANSION,
    dropout = DROPOUT,
    device = DEVICE
).to(DEVICE)

In [18]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 42,741,690 trainable parameters


In [19]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)
#model.apply(initialize_weights)

In [20]:
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [21]:
import time
import math


    
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    
    epoch_loss = 0
    iterations = 0
    for i, batch in tqdm(enumerate(iterator)):
        src = torch.transpose(batch[0], dim0 = 1, dim1 = 0)
        trg = torch.transpose(batch[1], dim0 = 1, dim1 = 0)
        
        # [sentence_len, batch_size]  -> [batch_size, sentence_len]
        
        optimizer.zero_grad()
        
        output = model(src, trg[:,:-1])
                
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
            
        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
                
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
            
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        epoch_loss += float(loss)
        iterations += 1
        # The Concept is, rather than deleting them, it's better to send them back to CPU
        # For properties that follow back propagation.
        # For those that aren't included, we simply, delete them.
        """"
        Move all the back_prop gradients into cpu, 
        then delete them"""
        src, trg = src.cpu(), trg.cpu()
        loss, output = loss.cpu(), output.cpu()
        del batch, src, trg, loss, output
        torch.cuda.empty_cache()

    return epoch_loss / iterations

def evaluate(model, iterator, criterion):
    model.eval()
    
    epoch_loss = 0
    iterations = 0
    with torch.no_grad():
    
        for i, batch in tqdm(enumerate(iterator)):

            src = torch.transpose(batch[0], dim0 = 1, dim1 = 0)
            trg = torch.transpose(batch[1], dim0 = 1, dim1 = 0)
            
            # [sentence_len, batch_size]  -> [batch_size, sentence_len]
            
            output = model(src, trg[:,:-1])
            
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            
            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]
            
            loss = criterion(output, trg)

            epoch_loss += float(loss)
            iterations += 1
            
            """Memory Management"""
            src ,trg = src.cpu(),trg.cpu()
            loss, output = loss.cpu(), output.cpu()
            del batch, src, trg, loss, output
            torch.cuda.empty_cache()
    return epoch_loss / iterations


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

#### Training the model


In [22]:
""" Loading a pretrained model, if you trained it before. """
try:
    model.load_state_dict(torch.load('model_dicts/en_ger_attention_model.pt'))
    print("model_loaded_successfully")
    
    best_valid_loss = evaluate(model, iter(valid_loader), criterion)
    print(f'| Test Loss: {best_valid_loss:.3f} | Test PPL: {math.exp(best_valid_loss):7.3f} |')
    
except:
    print("Model not loaded with preset")
    best_valid_loss = float('inf')


"""Number of epochs is number of training iterations"""
N_EPOCHS = 1
CLIP = 1    
    
for epoch in range(N_EPOCHS):
    train_loader = DataLoader(train_list, batch_sampler = batch_sampler(train_list), collate_fn=collate_batch)
    start_time = time.time()
    
    train_loss = train(model, iter(train_loader), optimizer, criterion, CLIP)
    valid_loss = evaluate(model, iter(valid_loader), criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'en_ger_attention_model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

0it [00:00, ?it/s]

model_loaded_successfully


27it [00:06,  3.95it/s]
0it [00:00, ?it/s]

| Test Loss: 2.662 | Test PPL:  14.321 |


6441it [18:36,  5.77it/s]
27it [00:05,  4.61it/s]

Epoch: 01 | Time: 18m 42s
	Train Loss: 1.757 | Train PPL:   5.796
	 Val. Loss: 2.725 |  Val. PPL:  15.253


#### Inference.

In [23]:
model.load_state_dict(torch.load('model_dicts/en_ger_attention_model.pt'))
test_loss = evaluate(model, iter(valid_loader), criterion)
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

27it [00:05,  4.71it/s]

| Test Loss: 2.670 | Test PPL:  14.444 |


In [24]:
def translate_sentence(sentence, src_tokenizer, src_vocab, trg_vocab, model, device,max_len = 50):
    model.eval()
    
    src_tokens = ['<BOS>'] + [token.text.lower() for token in src_tokenizer(sentence)] + ['<EOS>']
    src_indexes = [src_vocab[token] for token in src_tokens]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)

    trg_indexes = []
    trg_indexes.append(trg_vocab['<BOS>'])

    for i in range(max_len):
        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)
        
        with torch.no_grad():
            output = model(src_tensor,trg_tensor)
        
        pred_token = output.cpu().argmax(2)[:,-1]
        
        trg_indexes.append(pred_token)

        if pred_token == trg_vocab['<EOS>']:
            break
    
    trg_tokens = [trg_vocab.lookup_token(i) for i in trg_indexes]
    print(trg_tokens)
    """Memory Management"""
    output = output.cpu()
    del output
    return trg_tokens

In [25]:
translation = translate_sentence(sentence = "Fuck your mother",
                                 src_tokenizer = en_tokenizer,
                                 src_vocab = vocab_en, 
                                 trg_vocab = vocab_de,
                                 model = model,
                                 device = DEVICE,
                                 max_len = 30
                                )

['<BOS>', 'scheiß', 'ihre', 'mutter', 'an', '.', '\n', '<EOS>']
